# Efficient Sorting

In this section, we will pull together everything we have learned in this algorithms module. **Merge Sort** is a parallelizable, divide and conquer algorithm which achieves the optimal time for general sorting algorithms.